In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
import os


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
import os

def load_csv_files(directory_path: str, file_pattern: str) -> pd.DataFrame:
    """
    Carga todos los archivos CSV que coincidan con el patrón especificado y los combina en un único DataFrame.
    
    Args:
        directory_path (str): Ruta del directorio que contiene los archivos CSV
        file_pattern (str): Patrón para encontrar los archivos CSV
    
    Returns:
        pd.DataFrame: DataFrame combinado con todos los datos
    """
    # Lista para almacenar todos los DataFrames
    dfs = []
    
    # Construir la ruta completa
    full_path_pattern = os.path.join(directory_path, file_pattern)
    print(f"Buscando archivos en: {full_path_pattern}")
    
    # Buscar todos los archivos que coincidan con el patrón
    found_files = glob.glob(full_path_pattern)
    print(f"Archivos encontrados: {len(found_files)}")
    
    for filename in found_files:
        try:
            print(f"Cargando archivo: {filename}")
            # Cargar cada archivo CSV
            df = pd.read_csv(filename)
            # Agregar el nombre del archivo como referencia
            df['source_file'] = os.path.basename(filename)
            dfs.append(df)
        except Exception as e:
            print(f"Error al cargar el archivo {filename}: {str(e)}")
    
    # Combinar todos los DataFrames
    if not dfs:
        raise ValueError(f"No se encontraron archivos CSV para procesar en {full_path_pattern}")
    
    return pd.concat(dfs, ignore_index=True)

def clean_and_validate_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpia y valida los datos según los requisitos especificados.
    
    Args:
        df (pd.DataFrame): DataFrame original
        
    Returns:
        pd.DataFrame: DataFrame limpio y validado
    """
    print("Iniciando limpieza y validación de datos...")
    # Crear una copia para no modificar el original
    df_clean = df.copy()
    
    # Convertir fecha a datetime
    print("Convirtiendo fechas...")
    df_clean['fecha'] = pd.to_datetime(df_clean['fecha'], errors='coerce')
    
    # Eliminar registros con fechas inválidas
    invalid_dates = df_clean['fecha'].isna().sum()
    print(f"Fechas inválidas encontradas: {invalid_dates}")
    df_clean = df_clean.dropna(subset=['fecha'])
    
    # Manejar valores faltantes en monto
    null_amounts = df_clean['monto'].isna().sum()
    print(f"Montos nulos encontrados: {null_amounts}")
    df_clean['monto'] = df_clean['monto'].fillna(0)
    
    # Eliminar registros con tipo o asset_id vacíos
    null_types = df_clean['tipo'].isna().sum()
    null_assets = df_clean['asset_id'].isna().sum()
    print(f"Tipos nulos: {null_types}, Asset IDs nulos: {null_assets}")
    df_clean = df_clean.dropna(subset=['tipo', 'asset_id'])
    
    # Eliminar duplicados considerando todas las columnas
    duplicates = df_clean.duplicated().sum()
    print(f"Duplicados encontrados: {duplicates}")
    df_clean = df_clean.drop_duplicates()
    
    print(f"Registros finales después de limpieza: {len(df_clean)}")
    return df_clean

def create_time_windows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Crea ventanas de tiempo de 15 minutos y rellena los intervalos faltantes.
    
    Args:
        df (pd.DataFrame): DataFrame limpio
        
    Returns:
        pd.DataFrame: DataFrame con ventanas de tiempo completas
    """
    print("Creando ventanas de tiempo...")
    # Asegurar que los datos estén ordenados por fecha
    df = df.sort_values('fecha')
    
    # Redondear las fechas a intervalos de 15 minutos
    df['fecha_redondeada'] = df['fecha'].dt.round('15min')
    
    # Crear un índice de tiempo completo
    fecha_min = df['fecha_redondeada'].min()
    fecha_max = df['fecha_redondeada'].max()
    print(f"Rango de fechas: {fecha_min} a {fecha_max}")
    
    # Agrupar por las columnas necesarias y calcular agregaciones
    print("Agregando datos por ventanas de tiempo...")
    grouped = df.groupby(['fecha_redondeada', 'tipo', 'asset_id']).agg({
        'monto': ['sum', 'mean', 'count']
    }).reset_index()
    
    # Aplanar los nombres de las columnas
    grouped.columns = ['fecha', 'tipo', 'asset_id', 'monto_sum', 'monto_mean', 'monto_count']
    
    print(f"Ventanas de tiempo creadas: {len(grouped)}")
    return grouped

def process_wallet_transactions(input_directory: str, input_pattern: str, output_file: str):
    """
    Función principal que procesa las transacciones de wallet según los requisitos especificados.
    
    Args:
        input_directory (str): Directorio que contiene los archivos de entrada
        input_pattern (str): Patrón para encontrar los archivos de entrada
        output_file (str): Nombre del archivo de salida
    """
    try:
        # 1. Cargar datos
        print("\nCargando archivos...")
        df_combined = load_csv_files(input_directory, input_pattern)
        print(f"Total de registros cargados: {len(df_combined)}")
        
        # 2. Limpiar y validar datos
        print("\nLimpiando y validando datos...")
        df_clean = clean_and_validate_data(df_combined)
        
        # 3. Crear ventanas de tiempo y agregar datos
        print("\nProcesando ventanas de tiempo...")
        df_processed = create_time_windows(df_clean)
        
        # 4. Seleccionar y renombrar columnas finales
        df_final = df_processed[['fecha', 'tipo', 'asset_id', 'monto_sum']].copy()
        df_final = df_final.rename(columns={'monto_sum': 'monto'})
        
        # 5. Ordenar por fecha
        df_final = df_final.sort_values('fecha')
        
        # Asegurar que el directorio de salida existe
        output_dir = os.path.dirname(output_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # 6. Guardar resultado
        print(f"\nGuardando resultado en {output_file}...")
        df_final.to_csv(output_file, index=False)
        
        print("Procesamiento completado exitosamente.")
        
        # 7. Retornar estadísticas básicas
        stats = {
            'registros_originales': len(df_combined),
            'registros_procesados': len(df_final),
            'periodo_inicio': df_final['fecha'].min(),
            'periodo_fin': df_final['fecha'].max(),
            'tipos_unicos': df_final['tipo'].nunique(),
            'assets_unicos': df_final['asset_id'].nunique()
        }
        
        return stats
        
    except Exception as e:
        print(f"Error durante el procesamiento: {str(e)}")
        raise

if __name__ == "__main__":
    # Configuración
    INPUT_DIRECTORY = r"C:\Users\Nabucodonosor\Documents\BITLINK\Repositorios\explorer-wallets\algorand_reports"
    INPUT_PATTERN = "wallet_transactions_*.csv"
    OUTPUT_FILE = "wallet_transactions_consolidated.csv"
    
    # Asegurar que el directorio de entrada existe
    if not os.path.exists(INPUT_DIRECTORY):
        print(f"El directorio {INPUT_DIRECTORY} no existe.")
        print("Directorio actual:", os.getcwd())
        print("Contenido del directorio actual:", os.listdir())
    else:
        print(f"Directorio encontrado: {INPUT_DIRECTORY}")
        print("Contenido del directorio:", os.listdir(INPUT_DIRECTORY))
    
    # Ejecutar procesamiento
    try:
        stats = process_wallet_transactions(INPUT_DIRECTORY, INPUT_PATTERN, OUTPUT_FILE)
        print("\nEstadísticas del procesamiento:")
        for key, value in stats.items():
            print(f"{key}: {value}")
    except Exception as e:
        print(f"Error en la ejecución: {str(e)}")

In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Leer el archivo CSV
file_path = r"C:\Users\Nabucodonosor\Documents\BITLINK\Repositorios\explorer-wallets\wallet_transactions_consolidated.csv"
df = pd.read_csv(file_path)

# Convertir la columna fecha a datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Separar transacciones enviadas y recibidas
df_recibido = df[df['tipo'] == 'Recibido']
df_enviado = df[df['tipo'] == 'Enviado']

# Crear la figura con subplots
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Montos por Tipo de Transacción', 'Balance Acumulado'),
    vertical_spacing=0.15
)

# Añadir las líneas de transacciones
fig.add_trace(
    go.Scatter(
        x=df_recibido['fecha'],
        y=df_recibido['monto'],
        name='Recibido',
        mode='lines+markers',
        line=dict(color='green'),
        marker=dict(size=8)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df_enviado['fecha'],
        y=df_enviado['monto'],
        name='Enviado',
        mode='lines+markers',
        line=dict(color='red'),
        marker=dict(size=8)
    ),
    row=1, col=1
)

# Calcular y añadir el balance acumulado
df_sorted = df.sort_values('fecha')
df_sorted['balance_acumulado'] = df_sorted.apply(
    lambda row: row['monto'] if row['tipo'] == 'Recibido' else -row['monto'], 
    axis=1
).cumsum()

fig.add_trace(
    go.Scatter(
        x=df_sorted['fecha'],
        y=df_sorted['balance_acumulado'],
        name='Balance Acumulado',
        mode='lines',
        line=dict(color='blue', width=2)
    ),
    row=2, col=1
)

# Actualizar el diseño
fig.update_layout(
    title='Análisis de Transacciones de Wallet',
    height=900,
    showlegend=True,
    template='plotly_white'
)

# Actualizar ejes
fig.update_xaxes(title_text='Fecha', row=1, col=1)
fig.update_xaxes(title_text='Fecha', row=2, col=1)
fig.update_yaxes(title_text='Monto (ALGO)', row=1, col=1)
fig.update_yaxes(title_text='Balance Acumulado (ALGO)', row=2, col=1)

# Mostrar la figura
fig.show()

# Imprimir estadísticas básicas
print("\nEstadísticas básicas:")
print(f"Total de transacciones: {len(df)}")
print(f"Período: {df['fecha'].min()} a {df['fecha'].max()}")
print("\nResumen por tipo de transacción:")
print(df.groupby('tipo')['monto'].agg(['count', 'sum', 'mean', 'max']))


Estadísticas básicas:
Total de transacciones: 15516
Período: 2023-01-06 13:30:00 a 2024-12-27 19:00:00

Resumen por tipo de transacción:
          count           sum          mean           max
tipo                                                     
Enviado     176  2.246847e+09  1.276617e+07  3.000000e+08
Recibido  15340  3.535561e+09  2.304799e+05  4.000000e+08
